In [24]:
import cudaq
import numpy as np
import scipy
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [25]:
mnist = fetch_openml('mnist_784', version=1, cache=True)
data = mnist['data']
labels = np.array(mnist['target'], dtype=np.int8)
        

labels_zero = labels[labels==0] 
labels_one = labels[labels==1] 
binary_labels = np.hstack((labels_zero, labels_one))
digits_zero = data[labels==0]
digits_one = data[labels==1]
binary_digits = np.vstack((digits_zero, digits_one))
        
pca = PCA(n_components=4)
sc = StandardScaler()
binary_digits = sc.fit_transform(binary_digits)
data = pca.fit_transform(binary_digits)
data = (data-np.min(data))/(np.max(data)-np.min(data))


x_train, x_test, y_train, y_test = train_test_split(data, binary_labels, test_size=0.2)

x_train=x_train[0:500]
x_test=x_test[0:200]
y_train=y_train[0:500]
y_test=y_test[0:200]

In [20]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((500, 4), (200, 4), (500,), (200,))

In [26]:
cudaq.set_target("nvidia")

qubit_count = 4

@cudaq.kernel
def kernel(angles: list[float]):
    # Allocate a qubit that is initialised to the |0> state.
    qubit = cudaq.qvector(qubit_count)

    # Define gates and the qubits they act upon.
    for i in range(qubit_count):
        ry(angles[i], qubit[i])
    
    for i in range(qubit_count-1):
        x.ctrl(qubit[i], qubit[i+1])

# Our hamiltonian will be the Z expectation value of our qubit.
hamiltonian = cudaq.spin.z(0)


In [27]:
cost_values = []

def cost(parameters):
    expectation_value = cudaq.observe(kernel, hamiltonian, parameters).expectation()

    return expectation_value

def callback(xk):
    cost_values.append(cost(xk))


image_num=x_train.shape[0]
params_num=x_train.shape[1]
print('number of image', image_num)
print('num of parameters', params_num)

energy=[]
#replace 10 with image_num
for image in range (10):
    
    init_params=x_train[image]
    result_vqe=scipy.optimize.minimize(cost,init_params,method='COBYLA', callback=callback, tol=1e-5)
    print(result_vqe.fun)
    if result_vqe.fun<0:
        energy.append(1)
    else:
        energy.append(0)

print(energy)

number of image 500
num of parameters 4
-0.9999999469945057
-0.9999998380422002
-0.9999999352690816
-0.9999999295823354
-1.0000001747700762
-1.0000000144677035
-0.9999999017252038
-0.9999999548731211
-1.0000000856509716
-1.0000000274555463
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
